# Project 3, Wrangle OpenStreetMap Data

## Map Area

For this project, I chose where I currently live - San Mateo, California. It is in the middle of the Bay Area, and also a very quiet place to live. I want to see everything this city has to offer.

Link to San Mateo coordinates and download on OSM - [https://www.openstreetmap.org/export#map=13/37.5572/-122.3149](https://www.openstreetmap.org/export#map=13/37.5572/-122.3149)

In [30]:
import xml.etree.cElementTree as ET
import pprint
import re
import collections


# 1. Data Audit / Quick Checks

## Examining the XML File

#### Looking at the XML file, there are different tags such as "nodes" and "ways". Here's a list of them:

In [31]:
datafile = 'map'

def count_tags(filename):
    tagsDict = {}
    for event, element in ET.iterparse(filename):
        if element.tag not in tagsDict.keys():
            tagsDict[element.tag] = 1
        else:
            tagsDict[element.tag] += 1

    return tagsDict
    print tagsDict
    
count_tags(datafile)

{'bounds': 1,
 'member': 10857,
 'meta': 1,
 'nd': 701309,
 'node': 618225,
 'note': 1,
 'osm': 1,
 'relation': 882,
 'tag': 139074,
 'way': 71557}

#### Looking at the "K" attribute values in the XML file, we can see whether the attributes are okay or problematic:

In [32]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        # YOUR CODE HERE
        lower_re = lower.search(element.attrib['k'])
        lower_colon_re = lower_colon.search(element.attrib['k'])
        problemchars_re = problemchars.search(element.attrib['k'])
        if lower_re:
            keys["lower"] += 1
        elif lower_colon_re:
            keys["lower_colon"] += 1        
        elif problemchars_re:
            keys["problemchars"] += 1
        else:
            keys['other'] += 1
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

pprint.pprint(process_map(datafile))

{'lower': 102700, 'lower_colon': 34753, 'other': 1621, 'problemchars': 0}


# 2. Problems Encountered in the Map

### The main issue I encountered in the OpenStreetMap dataset is that the street and postal codes are not consistent from user to user.

In this part of the project, I will use Audit.py to audit and clean abbreviated and incorrect street names as well as postal codes. I will replace street abbreviations like "St.", "Blvd", "Ave.", Rd" to their full name - "Street", "Boulevard", "Avenue", and "Road". In the Audit.py file, I take the following steps:

* Create a list of expected street types that do not need to be cleaned
* If the last word of a street name does not match one of the expected street types, I store that entry in a dictionary
* Once I have a dictionary of street names that are unexpected, I use the update_name function to update the unexpected names to new names that make sense

I did similar things to validate the postal codes. If a postal code is not a 5 digit number, I use the update_postcode function to fix them

Quick examples of the inconsistencies and their fixes, using Audit.py file:

* Rollins Rd => Rollins Road
* Granada St => Granada Street
* Leslie => Leslie Street
* 94002-2121 => 94002
* 944023025 => 94402

# 3. Prepare to Import Data to Database

* The next step is to prepare data to be inserted into a database, using P3_Data_Prep.py file

* Once the OSM data transformed into csv files, I can load it into the sqlite database, using database.py file

# 4. Statistical Overview of the Dataset

### File Sizes:

In [33]:
import os
print 'The map file is {} MB'.format(os.path.getsize('map')/(10.**6))
print 'The nodes.csv file is {} MB'.format(os.path.getsize('nodes.csv')/(10.**6))
print 'The nodes_tags.csv file is {} MB'.format(os.path.getsize('nodes_tags.csv')/(10.**6))
print 'The ways.csv file is {} MB'.format(os.path.getsize('ways.csv')/(10.**6))
print 'The ways_tags.csv file is {} MB'.format(os.path.getsize('ways_tags.csv')/(10.**6))
print 'The ways_nodes.csv file is {} MB'.format(os.path.getsize('ways_nodes.csv')/(10.**6))
print 'The project3.db file is {} MB'.format(os.path.getsize('project3.db')/10.**6)

The map file is 131.760241 MB
The nodes.csv file is 52.755535 MB
The nodes_tags.csv file is 0.415798 MB
The ways.csv file is 4.312471 MB
The ways_tags.csv file is 4.243671 MB
The ways_nodes.csv file is 16.855967 MB
The project3.db file is 88.997888 MB


### Number of Unique Users:

In [45]:
import sqlite3
sqlite_file = 'project3.db'
con = sqlite3.connect(sqlite_file)
cur = con.cursor()

cur.execute("SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;")
print cur.fetchall()

[(407,)]


## Number of Nodes and Ways:

In [47]:
cur.execute("SELECT count(*) FROM nodes")
print "Nodes: {}".format(cur.fetchall())
cur.execute("SELECT COUNT(*) FROM ways")
print "Ways: {}".format(cur.fetchall())

Nodes: [(618225,)]
Ways: [(71557,)]


## Top 10 Contributing Users:

In [57]:
cur.execute("SELECT e.user, COUNT(*) AS num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
            GROUP BY e.user ORDER BY num DESC LIMIT 10;")
pprint.pprint(cur.fetchall())

[(u'abel801', 97168),
 (u'calfarome', 55721),
 (u'nikhilprabhakar', 46554),
 (u'Luis36995', 44129),
 (u'RichRico', 43368),
 (u'samely', 35351),
 (u'dannykath', 35123),
 (u'Jothirnadh', 34874),
 (u'saikabhi', 33000),
 (u'karitotp', 32934)]


## Common Amenities:

In [58]:
cur.execute("SELECT value, COUNT(*) AS num FROM nodes_tags WHERE key='amenity' \
            GROUP BY value ORDER BY num DESC LIMIT 10;")
pprint.pprint(cur.fetchall())


[(u'restaurant', 135),
 (u'school', 56),
 (u'bench', 53),
 (u'place_of_worship', 43),
 (u'fast_food', 38),
 (u'cafe', 34),
 (u'toilets', 30),
 (u'fuel', 29),
 (u'bank', 27),
 (u'atm', 16)]


## Popular Cuisines:

In [61]:
cur.execute("SELECT value,COUNT(*) AS num FROM nodes_tags JOIN (SELECT DISTINCT(id) \
            FROM nodes_tags WHERE value = 'restaurant') i ON nodes_tags.id = i.id\
            WHERE nodes_tags.key='cuisine'\
            GROUP BY nodes_tags.value\
            ORDER BY num DESC;")
pprint.pprint(cur.fetchall())

[(u'japanese', 12),
 (u'mexican', 12),
 (u'chinese', 11),
 (u'italian', 10),
 (u'pizza', 10),
 (u'asian', 6),
 (u'burger', 3),
 (u'greek', 2),
 (u'indian', 2),
 (u'sushi', 2),
 (u'vietnamese', 2),
 (u'Persian', 1),
 (u'Salad,_Sandwiches,_Juice_Bars_&_Smoothies', 1),
 (u'american', 1),
 (u'breakfast_brunch_french', 1),
 (u'crepe;sandwich;breakfast', 1),
 (u'filipino', 1),
 (u'fondue', 1),
 (u'hawaiian', 1),
 (u'ice_cream', 1),
 (u'international', 1),
 (u'japanese;korean', 1),
 (u'korean', 1),
 (u'mediterranean;middle_eastern', 1),
 (u'pancake', 1),
 (u'regional', 1),
 (u'sandwich', 1),
 (u'seafood', 1),
 (u'thai', 1)]


# 5. Conclusion

The OpenStreetMap data for San Mateo, California is quite an accomplishment that was achieved through having users contribute to the open source data. However, it is clear that the area is not complete nor very accurate. For example, in the "Popular Cuisines" query, Japanese Cuisines appeared multiple times - "japanese", "sushi", and "japanese;korean". I think a more standardized data entry should be implemented.

Also, based on the procedures to cleaning streets and postal codes, I noticed that not all streets are there, while others are either spelled wrong or have different variations of a single location.

## Suggestions:
* Have local business owners enter/edit their own locational data, since they have an incentive to keep their address and descriptions as accurate as possible
* Encourage more people to use OpenStreetMap and contribute to its content, will ensure a more complete picture of certain areas
* Build parsers or have designated moderators set regular cadence to check for data accuracy like street and zipcode inputs

### Benefits:
* If more people contribute to OpenStreetMap, then the user base will grow, and the data will be more complete, and we can have an true open source map that can be scraped and analyzed more easily
* If the data is more accurate, more people are inclined to use OpenStreetData and pay visits to the amenities, and thus generate more revenue for the city through tourism

### Anticipated Problems:
* OpenStreetMap is open source, which comes with the downside of not having paid employee to keep the data quality in check. Even if there are volunteer moderators to check the data every once in a while, there is no guarantee that the data is going to be clean and completely reliable
* It will take some time to truly get people to use and contribute to OpenStreetMap, since there are much better apps like Google Map. Awareness is key here. With such popularity, Google map almost completely overshadows OpenStreetMap, which means the general public and businesses most likely will not adopt OpenStreetMap as their primary source of locational data